In [65]:
from sklearn.datasets import load_wine
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

datasets = (load_wine())

x = pd.DataFrame(datasets["data"])
y = pd.DataFrame(datasets["target"])


In [66]:

print(x.shape,y.shape) # (178, 13) (178, 1)

# y의 클래스 값 분류 + 불균형확인
print(np.unique(y)) # [0 1 2]
print(np.unique(y,return_counts=True)) # [0 1 2]


(178, 13) (178, 1)
[0 1 2]
(array([0, 1, 2]), array([59, 71, 48], dtype=int64))


In [67]:
# print(x.info())
# print(x.describe())
# 데이터 분리
from sklearn.preprocessing import MinMaxScaler

x_train, x_test, y_train,y_test = train_test_split(x,y,
                                                   train_size=0.8,
                                                   shuffle = True,
                                                #  stratify 는 데이터 불균형을 해결해줌
                                                   stratify=y
                                                #    random_state=21
                                                   )
# x_train 을 스케일링
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
# x_test 를 스케일링
x_test = scaler.transform(x_test)


print(y_train[:5])
print("x_train shape : ",x_train.shape)
print("y_train shape : ",y_train.shape)

y_train = to_categorical(y_train)
print("to_cat_y_train shape : ",y_train.shape)



     0
154  2
57   0
41   0
2    0
54   0
x_train shape :  (142, 13)
y_train shape :  (142, 1)
to_cat_y_train shape :  (142, 3)


In [68]:
model = Sequential([
    Dense(10,activation="relu",input_shape=(13,)),
    Dense(10,activation="relu") ,
    Dense(10,activation="relu") ,
    Dense(10,activation="relu") ,
    Dense(10,activation="relu") ,
    Dense(10,activation="relu") ,
    # 다중분류모델의 활성화 함수는 softmax 입니다
    Dense(3,activation="softmax")
])

In [69]:
# 이진분류는 마지막 활성함수는 sigmoid + loss 는 바이너리 크로스 엔트로피 
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
monitor='val_loss',
min_delta=0.01, patience=10, 
verbose=1, 
mode='min')

# 훈련값이 int 형이기 때문에 sparse 를 사용합니다.
model.compile(loss="categorical_crossentropy"
              ,optimizer="adam"
              ,metrics=["accuracy"]
              )

hist = model.fit(x_train, y_train, epochs=100, batch_size=8, 
                 validation_split=0.2,
                 callbacks = [early_stopping])

# metrics 에 accuracy 사용가능


Epoch 1/100
15/15 [==============================] - 1s 14ms/step - loss: 8.6994 - accuracy: 0.4248 - val_loss: 6.3312 - val_accuracy: 0.3103
Epoch 2/100
15/15 [==============================] - 0s 6ms/step - loss: 3.7560 - accuracy: 0.4248 - val_loss: 2.6926 - val_accuracy: 0.3103
Epoch 3/100
15/15 [==============================] - 0s 5ms/step - loss: 1.6663 - accuracy: 0.4248 - val_loss: 1.3991 - val_accuracy: 0.3103
Epoch 4/100
15/15 [==============================] - 0s 5ms/step - loss: 1.1458 - accuracy: 0.4248 - val_loss: 1.0627 - val_accuracy: 0.3793
Epoch 5/100
15/15 [==============================] - 0s 5ms/step - loss: 0.9887 - accuracy: 0.6283 - val_loss: 0.9670 - val_accuracy: 0.6552
Epoch 6/100
15/15 [==============================] - 0s 5ms/step - loss: 0.9315 - accuracy: 0.6283 - val_loss: 0.9177 - val_accuracy: 0.6552
Epoch 7/100
15/15 [==============================] - 0s 5ms/step - loss: 0.9449 - accuracy: 0.6549 - val_loss: 0.8992 - val_accuracy: 0.6207
Epoch 8/100


In [70]:
y_test = to_categorical(y_test)
loss,accuracy = model.evaluate(x_test,y_test)

print("loss:",loss,"\nacc :" ,accuracy)


2/2 [==============================] - 0s 4ms/step - loss: 1.1054 - accuracy: 0.2778
loss: 1.1054004430770874 
acc : 0.2777777910232544


In [71]:
from sklearn.metrics import accuracy_score
y_predict = model.predict(x_test)

# predict 의 값은 총 3개가 나오는데 softmax 특성상 sum 의 값은 1이 됨
print(y_predict[:5])

# argmax 는 x 를 최대값으로 만들어주는 입력을 구하는함수
y_predict = np.argmax(y_predict,axis=1)
# y_predict = to_categorical(y_predict,3)
print("예측 : ", y_predict[:15])
# y_test 를 원핫 인코딩 해제 해야함
y_test = np.argmax(y_test,axis=1)
print("실제 : ", y_test[:15])

print
print("========================================")
acc = accuracy_score(y_test,y_predict)
print("acc : ",acc)

[[0.3123032  0.34917244 0.33852437]
 [0.3112411  0.34604526 0.34271365]
 [0.30911326 0.34645534 0.3444314 ]
 [0.309876   0.34670246 0.34342155]
 [0.31193092 0.3472521  0.340817  ]]
예측 :  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 2]
실제 :  [2 0 0 0 1 1 2 0 0 1 1 1 2 2 1]
acc :  0.2777777777777778
